In [2]:
import numpy as np
from numpy import dot
from scipy.linalg import inv

In [3]:
# step 1 desing the state, create the vector that will stor the n states. in this case you have position and velocity. 

position_x = []
velocity_x_dot = []
# thus
state_bold_x = np.array([[position_x],
                        [velocity_x_dot]])

In [4]:
# step 1.1 design the state covariance. 
# the variance sneed to be reasonable values. Recall that this matrix has the variances (sigman**2), also that position and velocity are correclated. 
# if you know how correlated they are good use the number, otherwise
# if you have no idea how correlated they are dont worry the filter will compute this. 
sigma_squared_x_initial = []
sigma_squared_x_dot_initial = []
# this
P = np.diag([sigma_squared_x_initial, sigma_squared_x_dot_initial])

In [5]:
# now you need to specfifyt the state tranisiton function (F)
# our job as kalman filter desinger is to spefiy F usch that xhat = Fx peforms the innovation (prediction) for our system. 
# design the process model. Here youre going to use the ower of linear algebra. 
# each variable in the state vector (x) needs an innovation equation. 
# for position_x assume the innovation equation is position_x + (velocity_x_dot * dt)
# assume velocity is not know so velocity_x_dot_hat = velocity_x_dot . this holds true as long as velocity doens tchange too much over each innovation . 




# process model looks like               xHat   =   position_x + velocity_x_dot* dt
#                          velocity_x_dot_hat   =       0      + velocity_x_dot 

# thus the state transition function is            [[  1    ,    dt] 
#                                                   [  0    ,    1 ]]

In [16]:
dt = 0.1
x = np.array([10.0, 4.5])
P = np.diag([500, 49])
F = np.array([[1,dt],
              [0,1]])
B = np.array([])
u = np.array([])
Q = np.array([[0, 0],
             [0, 0]])

def innovation(x, P, F):
    
    x = dot(F, x) + dot(B, u)
    P = dot(dot(F,P),F.T) + Q
    
#     print(x)
#     print(P)
    
    return x, P

In [17]:
innovation(x, P, F)

(array([10.45,  4.5 ]), array([[500.49,   4.9 ],
        [  4.9 ,  49.  ]]))

In [18]:
dt = 0.1
x = np.array([10.0, 4.5])
P = np.diag([500, 49])
F = np.array([[1,dt],
              [0,1]])

x, P = innovation(x=x, P=P, F=F)
print('x = ', x)

('x = ', array([10.45,  4.5 ]))


In [19]:

for _ in range (4):
    x ,P = innovation(x=x, P=P, F=F)
    print('x = ', x)

('x = ', array([10.9,  4.5]))
('x = ', array([11.35,  4.5 ]))
('x = ', array([11.8,  4.5]))
('x = ', array([12.25,  4.5 ]))


In [20]:
# innovation() computes both the mean and covariance of the innovation.
print(P)

[[512.25  24.5 ]
 [ 24.5   49.  ]]


In [21]:
# the value avove is the  value of P after five innovation (prediction)
# we denote this as PHat in the kalman filter equations 

In [22]:
def predict(x, P, F, B, u, Q):
    
    x = dot(F, x) + dot(B, u)
    P = dot(dot(F,P),F.T) + Q
    
#     print(x)
#     print(P)
    
    return x, P



def update(H,P, R, z):
    S = dot(H,P).dot(H.T) + R    #  S = dot(dot(H,P),H.T)  #system uncertainty
    K = dot(P, H.T).dot(inv(S))  #  K = dot(dot(P,H.T),inv(S)) Kalman gain 
    y = z - dot(H,x)
    x += dot(K,y)   # x = x + dot(K,y)  state update
    P = P - dot(K, H).dot(P)  # P = P - dot(dot(K,H),P)  covariance update
    
    return x, P

In [29]:
class KalmanFilter(object):
    
    def __init__(self, states_x, measurements_z, control_input_u=0):
        
        self.states_x = states_x
        self.measurements_z = measurements_z
        self.control_input_u = control_input_u
        
        self.x = np.zeros((states_x, 1))    # states
        self.P = np.eye(states_x)           # uncertainty covariance
        self.Q = np.eye(states_x)
        self.B = None
        self.F = np.eye(states_x)
        self.H = np.zeros((measurements_z, states_x))
        self.R = np.eye(measurements_z)
        self.z = np.array([[None]*self.measurements_z]).T
        
        self.K = np.zeros((states_x, measurements_z)) # kalman gain
        self.y = np.zeros((measurements_z, 1))
        self.S = np.zeros((states_x, measurements_z)) # system uncertainty
        self.SI = np.zeros((measurements_z, measurements_z)) # inverse system uncertainty
        
        # identiy matrix. Do no talter this 
        self._I = np.eye(states_x)
        
        self.inv = np.linalg.inv
        
    def predict(self, u=None, F=None, Q=None):
        
        x = dot(F, x) + dot(B, u)
        P = dot(dot(F,P),F.T) + Q
        
        self.x_prior = self.x.copy()
        self.P_prior = self.P.copy()
        
        
    def update(self, z, R=None, H=None):
        
        self.S = dot(H,dot(self.P,H.T)) + R
        self.SI = self.inv(self.S)
        self.K = dot(dot(self.P,H.T),self.SI)
        
        # x = x + Ky
        # preditct new x with residual scaled by the kalman gain
        self.x = xelf.x + dot(self.K, self.y)
        
        # predict new covariance matrix
        I_KH = self._I - dot(self.K, H)
        self.P = dot(dot(I_KH, self.P), I_KH.T) + dot(dot(self.K, R), self.K.T)
        
        self.x_post = self.x.copy()
        self.P_post = self.P.copy()

        
        
        

In [34]:
kf = KalmanFilter(states_x=2, measurements_z=1)

In [41]:

dt = 0.1
x = np.array([10.0, 4.5])
P = np.diag([500, 49])
F = np.array([[1,dt],
              [0,1]])
B = np.array([])
u = np.array([])
Q = np.array([[0, 0],
             [0, 0]])


from numpy import dot
from scipy.linalg import inv

count = 50
# track, zs = compute_dog_data(R_var, Q_var, count)
xs, cov = [], []
for z in zs:
    # predict
    x = dot(F, x)
    P = dot(F, P).dot(F.T) + Q
    
    #update
    S = dot(H, P).dot(H.T) + R
    K = dot(P, H.T).dot(inv(S))
    y = z - dot(H, x)
    x += dot(K, y)
    P = P - dot(K, H).dot(P)
    
           self.x = xelf.x + dot(self.K, self.y)
        
        # predict new covariance matrix
        I_KH = self._I - dot(self.K, H)
        self.P = dot(dot(I_KH, self.P), I_KH.T) + dot(dot(self.K, R), self.K.T)
        
    
    xs.append(x)
    cov.append(P)

xs, cov = np.array(xs), np.array(cov)
# plot_track(xs[:, 0], track, zs, cov, plot_P=False, dt=dt)

